# Загрузка библиотек

In [2]:
import pandas as pd
import re
import string
import nltk
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, f1_score, precision_score, recall_score
import joblib
import numpy as np

# === 1. Загрузка данных ===

In [3]:
def advanced_clean_text(text):
    text = text.lower()
    
    # Удаление email адресов
    text = re.sub(r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b', ' ', text)
    # Удаление URL
    text = re.sub(r'http\S+|www\.\S+', ' ', text)
    # Удаление телефонных номеров
    text = re.sub(r'\b\d{10,}\b', ' ', text)
    # Удаление денежных сумм (но сохраняем символ валюты как признак спама)
    text = re.sub(r'\$\d+', ' money ', text)
    # Удаление обычных чисел
    text = re.sub(r'\b\d+\b', ' ', text)
    # Сохраняем восклицательные знаки (признак спама)
    text = re.sub(r'!+', ' excl ', text)
    # Удаление пунктуации (кроме некоторых символов)
    text = re.sub(r'[^\w\s!?]', ' ', text)
    # Удаление лишних пробелов
    text = re.sub(r'\s+', ' ', text).strip()
    return text

# === 2. Очистка текста ===

In [4]:
# Загрузка и подготовка данных
print("Загрузка данных...")
data = pd.read_csv("spam.csv", encoding="latin-1")
data = data[['v1', 'v2']]
data.columns = ['label', 'message']

print("Распределение классов:")
print(data['label'].value_counts())

data['label'] = data['label'].map({'ham': 0, 'spam': 1})

# Применяем улучшенную очистку
print("Предобработка текста...")
data['cleaned_message'] = data['message'].apply(advanced_clean_text)

# Удаляем слишком короткие сообщения
data['message_length'] = data['cleaned_message'].apply(len)
data = data[data['message_length'] > 3]

Загрузка данных...
Распределение классов:
label
ham     4825
spam     747
Name: count, dtype: int64
Предобработка текста...


# Оптимизация KNN

In [5]:
X = data['cleaned_message']
y = data['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print(f"\nРазмер тренировочной выборки: {len(X_train)}")
print(f"Размер тестовой выборки: {len(X_test)}")


Размер тренировочной выборки: 4436
Размер тестовой выборки: 1109


# === Обучение ===

In [6]:
# Расширенный поиск лучших параметров для KNN
print("\nПоиск оптимальных параметров KNN...")

knn_pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('knn', KNeighborsClassifier())
])

# Расширенная сетка параметров
param_grid = {
    'tfidf__max_features': [3000, 5000, 8000, 10000],
    'tfidf__ngram_range': [(1, 1), (1, 2), (1, 3)],
    'tfidf__min_df': [1, 2, 3],
    'tfidf__max_df': [0.7, 0.8, 0.9],
    'tfidf__stop_words': [None, 'english'],
    'tfidf__sublinear_tf': [True, False],
    'knn__n_neighbors': [3, 5, 7, 9, 11, 15],
    'knn__metric': ['cosine', 'euclidean', 'manhattan', 'minkowski'],
    'knn__weights': ['uniform', 'distance'],
    'knn__algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute']
}


Поиск оптимальных параметров KNN...


# === 5. Обучение k-NN ===

In [7]:
# Используем RandomizedSearchCV для более быстрого поиска
from sklearn.model_selection import RandomizedSearchCV

knn_search = RandomizedSearchCV(
    knn_pipeline, 
    param_grid, 
    n_iter=50,  # Проверяем 50 случайных комбинаций
    cv=3, 
    scoring='f1',
    n_jobs=-1,
    random_state=42
)

knn_search.fit(X_train, y_train)

print("Лучшие параметры найдены!")
print("Лучшие параметры:")
for param, value in knn_search.best_params_.items():
    print(f"  {param}: {value}")

Лучшие параметры найдены!
Лучшие параметры:
  tfidf__sublinear_tf: False
  tfidf__stop_words: None
  tfidf__ngram_range: (1, 1)
  tfidf__min_df: 2
  tfidf__max_features: 5000
  tfidf__max_df: 0.9
  knn__weights: distance
  knn__n_neighbors: 3
  knn__metric: cosine
  knn__algorithm: auto


c:\Users\tybfist\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\model_selection\_validation.py:516: FitFailedWarning: 
18 fits failed out of a total of 150.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
9 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\tybfist\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\model_selection\_validation.py", line 859, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
    ~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\tybfist\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\base.py", line 1365, in wrapper
    return fit_method(estimator, *args, **kwargs)
  File "c:\Users\tybfis

# === Сохранение модели ===

In [8]:
# Обучение финальной модели с лучшими параметрами
final_knn = knn_search.best_estimator_

# Детальная оценка
y_pred = final_knn.predict(X_test)
y_pred_proba = final_knn.predict_proba(X_test)

print("\n" + "="*60)
print("ФИНАЛЬНЫЕ РЕЗУЛЬТАТЫ KNN")
print("="*60)

print(f"Accuracy: {accuracy_score(y_test, y_pred):.4f}")
print(f"Precision: {precision_score(y_test, y_pred):.4f}")
print(f"Recall: {recall_score(y_test, y_pred):.4f}")
print(f"F1-score: {f1_score(y_test, y_pred):.4f}")

print("\nClassification Report:")
print(classification_report(y_test, y_pred))

print("\nМатрица ошибок:")
print(confusion_matrix(y_test, y_pred))


ФИНАЛЬНЫЕ РЕЗУЛЬТАТЫ KNN
Accuracy: 0.9793
Precision: 0.9846
Recall: 0.8591
F1-score: 0.9176

Classification Report:
              precision    recall  f1-score   support

           0       0.98      1.00      0.99       960
           1       0.98      0.86      0.92       149

    accuracy                           0.98      1109
   macro avg       0.98      0.93      0.95      1109
weighted avg       0.98      0.98      0.98      1109


Матрица ошибок:
[[958   2]
 [ 21 128]]


# Сохранение векторизатора отдельно (для возможности использования отдельно)

In [9]:
# Анализ вероятностей для лучшего понимания модели
print("\nАнализ вероятностей:")
spam_probs = y_pred_proba[y_test == 1, 1]  # Вероятности для реального спама
ham_probs = y_pred_proba[y_test == 0, 1]   # Вероятности для реального не-спама

print(f"Средняя вероятность спама для реального спама: {spam_probs.mean():.3f}")
print(f"Средняя вероятность спама для реального не-спама: {ham_probs.mean():.3f}")


Анализ вероятностей:
Средняя вероятность спама для реального спама: 0.849
Средняя вероятность спама для реального не-спама: 0.009


In [10]:
# Сохранение модели
joblib.dump(final_knn, 'improved_knn_spam_model.pkl')
print("\nМодель сохранена как 'improved_knn_spam_model.pkl'")


Модель сохранена как 'improved_knn_spam_model.pkl'


In [11]:
# Функция для проверки конкретных сообщений
def analyze_message(model, message):
    cleaned = advanced_clean_text(message)
    prediction = model.predict([cleaned])[0]
    probability = model.predict_proba([cleaned])[0]
    
    return {
        'original': message,
        'cleaned': cleaned,
        'prediction': 'SPAM' if prediction == 1 else 'HAM',
        'spam_probability': probability[1],
        'ham_probability': probability[0]
    }

In [ ]:
# Тестирование на характерных примерах

test_messages = [
    "Congratulations! You won $1000 prize! Call now to claim!!!",
    "Hey, are we still meeting for lunch tomorrow?",
    "URGENT: Your bank account needs verification. Click here.",
    "Hi mom, I'll be home late today. See you for dinner",
    "FREE iPhone waiting for you! Claim your prize now!",
    "Meeting rescheduled to 3 PM. Please confirm attendance.",
    "WINNER!! You have been selected for a free gift!",
    "Can you pick up milk on your way home?"
]


ТЕСТИРОВАНИЕ НА ПРИМЕРАХ


In [13]:
for msg in test_messages:
    result = analyze_message(final_knn, msg)
    print(f"\nСообщение: {result['original']}")
    print(f"Очищенное: {result['cleaned']}")
    print(f"Результат: {result['prediction']} (вероятность спама: {result['spam_probability']:.3f})")



Сообщение: Congratulations! You won $1000 prize! Call now to claim!!!
Очищенное: congratulations excl you won money prize excl call now to claim excl
Результат: SPAM (вероятность спама: 0.659)

Сообщение: Hey, are we still meeting for lunch tomorrow?
Очищенное: hey are we still meeting for lunch tomorrow?
Результат: HAM (вероятность спама: 0.000)

Сообщение: URGENT: Your bank account needs verification. Click here.
Очищенное: urgent your bank account needs verification click here
Результат: HAM (вероятность спама: 0.327)

Сообщение: Hi mom, I'll be home late today. See you for dinner
Очищенное: hi mom i ll be home late today see you for dinner
Результат: HAM (вероятность спама: 0.000)

Сообщение: FREE iPhone waiting for you! Claim your prize now!
Очищенное: free iphone waiting for you excl claim your prize now excl
Результат: HAM (вероятность спама: 0.319)

Сообщение: Meeting rescheduled to 3 PM. Please confirm attendance.
Очищенное: meeting rescheduled to pm please confirm attendance

In [14]:
# Интерактивный режим проверки
def interactive_check():
    print("\n" + "="*50)
    print("ИНТЕРАКТИВНАЯ ПРОВЕРКА")
    print("Введите сообщение для проверки (или 'quit' для выхода):")
    
    while True:
        user_input = input("\nСообщение: ")
        if user_input.lower() == 'quit':
            break
            
        if user_input.strip():
            result = analyze_message(final_knn, user_input)
            prob = result['spam_probability']
            
            if prob > 0.7:
                print(f"🚫 ВЫСОКАЯ ВЕРОЯТНОСТЬ СПАМА ({prob:.1%})")
            elif prob > 0.3:
                print(f"⚠️  ВОЗМОЖНО СПАМ ({prob:.1%})")
            else:
                print(f"✅ ВЕРОЯТНО НЕ СПАМ ({prob:.1%})")

# Запуск интерактивной проверки
interactive_check()


ИНТЕРАКТИВНАЯ ПРОВЕРКА
Введите сообщение для проверки (или 'quit' для выхода):
✅ ВЕРОЯТНО НЕ СПАМ (0.0%)
🚫 ВЫСОКАЯ ВЕРОЯТНОСТЬ СПАМА (100.0%)
✅ ВЕРОЯТНО НЕ СПАМ (0.0%)
